## Causal Learning notebook for Break-Cancer Diagnosis

### Importing necessary packages, and libraries

In [1]:
import logging
import logging.handlers
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
from causalgraphicalmodels import CausalGraphicalModel

import os
import sys

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from project_config import Config
from FileHandler import FileHandler
from log import logger
from data_processor import data_processor
from plot import Plot

## Creating Script Class Objects

In [2]:
file = FileHandler()
log = logger('eda_logs')
# plot = plot()
file_config = Config()
data_processor = data_processor()
plot = Plot()

## Loading the Dataset

In [3]:
# print(Config.Data_path)
df = file.csv_file_read("processed_data")
print("Dataset columns are: ", df.columns)
df.info()

Dataset columns are:  Index(['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   diagnosis                569 non-null    object 
 1   radius_mean              569 non-null    float64
 2   texture_

## Drawing the Causal graph

In [6]:
#area and perimeter functions of radius
from causalgraphicalmodels import CausalGraphicalModel
causal = CausalGraphicalModel(
    nodes=['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean',
           'concave points_mean','symmetry_mean','fractal_dimension_mean','diagnosis'],
    edges=[
        ("radius_mean", "perimeter_mean"), 
        ("radius_mean", "area_mean"),
        ("radius_mean","smoothness_mean"),
        ("area_mean","compactness_mean"),
        ("perimeter_mean","compactness_mean"),
        ("concave points_mean","concavity_mean"),
        ("smoothness_mean","diagnosis"),
        ("concave points_mean","diagnosis"),
        ("concavity_mean","diagnosis"),
        ("compactness_mean","diagnosis"),
        ("perimeter_mean", "diagnosis"),
        ("area_mean", "diagnosis"),
        ("symmetry_mean", "diagnosis"),
        ("fractal_dimension_mean", "diagnosis"),
        ("texture_mean", "diagnosis")
    ]
)

In [ ]:
causal.draw()